## Update to model registry

Use the regular Python 3 kernel

In [ ]:
# MLflow experiment and model registry name
EXPERIMENT_NAME = "yolov8n_new_xyz"
REGISTERED_MODEL_NAME = "yolov8n_test"

In [ ]:
from pathlib import Path
import mlflow

# ==========================
# Config (edit as needed)
# ==========================
# Folder that contains model.onnx and config.pbtxt
SRC_DIR = Path(f"models/{REGISTERED_MODEL_NAME}")



# Optional: set tracking URI if you're not using local ./mlruns
# mlflow.set_tracking_uri("http://your-mlflow-server:5000")

# ==========================
# Sanity checks
# ==========================
if not SRC_DIR.exists():
    raise FileNotFoundError(f"Source directory not found: {SRC_DIR}")

onnx_path = SRC_DIR / "model.onnx"
config_path = SRC_DIR / "config.pbtxt"

if not onnx_path.exists():
    raise FileNotFoundError(f"ONNX file not found: {onnx_path}")
if not config_path.exists():
    raise FileNotFoundError(f"config.pbtxt not found: {config_path}")

print(f"Using source directory: {SRC_DIR.resolve()}")

# ==========================
# Set / create experiment
# ==========================
mlflow.set_experiment(EXPERIMENT_NAME)

# ==========================
# Start run, log artifacts
# ==========================
with mlflow.start_run(run_name="yolov8n-triton-bundle") as run:
    run_id = run.info.run_id
    print(f"MLflow run_id: {run_id}")

    # This copies the ENTIRE folder models/yolov8n/*
    # into artifacts/model/
    mlflow.log_artifacts(str(SRC_DIR), artifact_path="model")

    # Build model URI pointing to the "model" artifact folder
    model_uri = f"runs:/{run_id}/model"
    print(f"Model artifacts logged at: {model_uri}")

    # ==========================
    # Register the model
    # ==========================
    registration = mlflow.register_model(
        model_uri=model_uri,
        name=REGISTERED_MODEL_NAME,
    )

print("========== Registration Complete ==========")
print(f"Registered model name   : {REGISTERED_MODEL_NAME}")
print(f"Registered model version: {registration.version}")
print(f"Source run_id           : {run_id}")
print(f"Source artifacts URI    : {model_uri}")


In [ ]:
# ==========================
# Config – edit as needed
# ==========================
MODEL_VERSION = registration.version  # or "1", or whatever version you want to pull
#MODEL_VERSION = 1  # or "1", or whatever version you want to pull

In [ ]:
## Download from model registry

In [ ]:
from pathlib import Path
import shutil
import mlflow



# Where you want the final layout written
OUTPUT_ROOT = Path("downloaded-models")

# Optional: set tracking URI if you're not using local ./mlruns
# mlflow.set_tracking_uri("http://your-mlflow-server:5000")

# ==========================
# Build MLflow model URI
# ==========================
model_uri = f"models:/{REGISTERED_MODEL_NAME}/{MODEL_VERSION}"
print(f"Downloading artifacts from: {model_uri}")

# This downloads the registered model's artifacts to a local temp dir
local_dir = mlflow.artifacts.download_artifacts(artifact_uri=model_uri)
local_dir = Path(local_dir)
print(f"Downloaded to temp dir: {local_dir}")

# ==========================
# Find config.pbtxt and .onnx
# ==========================
config_files = list(local_dir.rglob("config.pbtxt"))
onnx_files = list(local_dir.rglob("*.onnx"))

if not config_files:
    raise FileNotFoundError(f"No config.pbtxt found under {local_dir}")
if not onnx_files:
    raise FileNotFoundError(f"No .onnx files found under {local_dir}")

config_src = config_files[0]         # first match
onnx_src = onnx_files[0]             # first match (adjust if you expect multiple)

print(f"Found config: {config_src}")
print(f"Found ONNX  : {onnx_src}")

# ==========================
# Build desired output layout
# ==========================

OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

model_root_dir = OUTPUT_ROOT / REGISTERED_MODEL_NAME
version_dir = model_root_dir / str(MODEL_VERSION)


model_root_dir.mkdir(parents=True, exist_ok=True)
version_dir.mkdir(parents=True, exist_ok=True)

# Destinations:
config_dest = model_root_dir / "config.pbtxt"
onnx_dest = version_dir / onnx_src.name  # keep same filename (e.g., model.onnx)

# ==========================
# Copy files into place
# ==========================
shutil.copy2(config_src, config_dest)
shutil.copy2(onnx_src, onnx_dest)

print("=========== Done ===========")
print(f"Config copied to : {config_dest}")
print(f"ONNX copied to   : {onnx_dest}")
print()
print("Final structure:")
print(str(OUTPUT_ROOT))
print(f"  └─ {REGISTERED_MODEL_NAME}/")
print(f"       ├─ config.pbtxt")
print(f"       └─ {MODEL_VERSION}/")
print(f"            └─ {onnx_src.name}")


## Copy to the shared EDV to deploy to Triton

In [ ]:
from pathlib import Path
import shutil
import os
# -----------------------------
# Config – edit these
# -----------------------------

SOURCE_ROOT = Path(f"./downloaded-models")                  # where your model folder exists
TARGET_ROOT = Path("/domino/edv/domino-inference-test-triton-inference-server-pvc")        # where you want to copy the versioned folder
#TARGET_ROOT = Path("/domino/edv/triton-management-rw-dev-pvc")        # where you want to copy the versioned folder


# -----------------------------
# Build paths
# -----------------------------
src_config_file = SOURCE_ROOT / REGISTERED_MODEL_NAME / "config.pbtxt"
dst_config_file = TARGET_ROOT / REGISTERED_MODEL_NAME / "config.pbtxt"
src_version_dir = SOURCE_ROOT / REGISTERED_MODEL_NAME / str(MODEL_VERSION)
dst_version_dir = TARGET_ROOT / REGISTERED_MODEL_NAME / str(MODEL_VERSION)


# Ensure source exists
if not src_version_dir.exists():
    raise FileNotFoundError(f"Source version directory not found: {src_version_dir}")

# Create destination root
dst_version_dir.mkdir(parents=True, exist_ok=True)

def copy_file_simple(src: Path, dst: Path):
    """Copy file contents only, no metadata, safe for S3/fuse."""
    dst.parent.mkdir(parents=True, exist_ok=True)
    with src.open("rb") as fsrc, dst.open("wb") as fdst:        
        shutil.copyfileobj(fsrc, fdst)
        print(f"Copied {src} -> {dst}")

#Copy config.pbtxt
copy_file_simple(src_config_file,dst_config_file)



# -----------------------------
# Recursively copy tree
# -----------------------------

for root, dirs, files in os.walk(src_version_dir):
    root_path = Path(root)
    rel = root_path.relative_to(src_version_dir)
    current_dst_root = dst_version_dir / rel

    # Ensure subdirs exist at destination
    for d in dirs:
        (current_dst_root / d).mkdir(parents=True, exist_ok=True)

    # Copy files without metadata
    for f in files:
        
        src_file = root_path / f
        dst_file = current_dst_root / f
        copy_file_simple(src_file, dst_file)

print("========== Done ==========")
print(f"Copied version folder:")
print(f"  {src_version_dir}  →  {dst_version_dir}")